<a href="https://colab.research.google.com/github/ifeLight/ml-bot/blob/main/binance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-ta
!pip install backtrader
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=724a7eaa4a653bbf2e2a3a759fd242c0a3b908b0edd3ba526a6e4664e07739f1
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import requests
import json
import datetime
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import backtrader as bt
import pandas_ta as ta

In [3]:

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

binance_base_url = 'https://52on3577u3.execute-api.eu-central-1.amazonaws.com'

def get_binance_candles(symbol: str, interval='1h', limit: int = 50, **kwargs):
    url = f'{binance_base_url}/api/v3/uiKlines?symbol={symbol}&interval={interval}&limit={limit}'
    for key, value in kwargs.items():
        url += f'&{key}={value}'
    response = requests.get(url)
    result = json.loads(response.text)
    # print(result)
    def map_result(x):
        return {
            'Date': x[0],
            'Open': x[1],
            'High': x[2],
            'Low': x[3],
            'Close': x[4],
            'Volume': x[5],
        }
    mappeded_result = []
    for x in result:
        mappeded_result.append(map_result(x))
    return mappeded_result


def candles_to_df(data):
    df =  pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'], unit='ms')
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    df.set_index('Date', inplace=True)
    return df

def get_all_binance_candles(symbol: str, interval='1h', start_date=None, end_date=None, limit=1000):
  result = []
  start_date = pd.to_datetime(start_date) if start_date else pd.to_datetime('2015-01-01')
  end_date = pd.to_datetime(end_date) if end_date else pd.to_datetime('today')
  while True:
    candles = get_binance_candles(symbol, interval, limit, startTime=int(start_date.timestamp() * 1000), endTime=int(end_date.timestamp() * 1000))
    if len(candles) <= 1:
      break;
    result += candles
    start_date = pd.to_datetime(datetime.datetime.fromtimestamp(candles[-1]['Date'] / 1000))
  return candles_to_df(result)

In [4]:
data = get_all_binance_candles('BTCUSDT', '4h', '01-01-2024', '31-12-2024')
data

<ipython-input-3-63ee79f3f45f>:42: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  end_date = pd.to_datetime(end_date) if end_date else pd.to_datetime('today')


,Open,High,Low,Close,Volume
Date,,,,,
2024-01-01 00:00:00,42283.58,42775.00,42230.08,42330.49,3948.08335
2024-01-01 04:00:00,42330.50,42500.00,42180.77,42492.46,2706.99880
2024-01-01 08:00:00,42492.46,42762.39,42452.58,42690.20,2948.80274
2024-01-01 12:00:00,42690.21,42847.07,42580.00,42783.05,2936.44406
2024-01-01 16:00:00,42783.05,43550.00,42664.42,43517.99,5686.97164
...,...,...,...,...,...
2024-12-30 08:00:00,93731.66,94068.56,93469.56,93923.14,1683.90269
2024-12-30 12:00:00,93923.14,94277.23,91530.45,92130.21,9295.96250
2024-12-30 16:00:00,92130.21,94684.09,91800.00,94417.04,6130.98017


In [5]:
def add_pivots(df, window=5):
    """
    Calculate the probability of price going up and down based on higher and lower pivots.
    """
    df['Higher_Pivot'] = df['High'].rolling(window=2 * window + 1, center=True).apply(
        lambda x: 1 if x.iloc[window] == x.max() else 0, raw=False
    )
    df['Lower_Pivot'] = df['Low'].rolling(window=2 * window + 1, center=True).apply(
        lambda x: 1 if x.iloc[window] == x.min() else 0, raw=False
    )

    # Step 2: Ensure no two successive pivots of the same type
    pivot_type = None  # Tracks the type of the last pivot
    last_pivot_index = None  # Tracks the index of the last pivot

    # Remove duplicated index
    df = df[~df.index.duplicated(keep='first')]

    for i in range(len(df.index)):
      if df.loc[df.index[i], 'Higher_Pivot'].item() == 1:
          if pivot_type == 'higher':
              # Compare current higher pivot with the last higher pivot
              if df.loc[df.index[i], 'High'].item() > df.loc[df.index[last_pivot_index], 'High'].item():
                  # Remove the last higher pivot
                  df.loc[df.index[last_pivot_index], 'Higher_Pivot'] = 0
                  # Update the last pivot
                  last_pivot_index = i
              else:
                  # Remove the current higher pivot
                  df.loc[df.index[i], 'Higher_Pivot'] = 0
          else:
              # Update pivot type and index
              pivot_type = 'higher'
              last_pivot_index = i

      elif df.loc[df.index[i], 'Lower_Pivot'].item() == 1:
          if pivot_type == 'lower':
              # Compare current lower pivot with the last lower pivot
              if df.loc[df.index[i], 'Low'].item() < df.loc[df.index[last_pivot_index], 'Low'].item():
                  # Remove the last lower pivot
                  df.loc[df.index[last_pivot_index], 'Lower_Pivot'] = 0
                  # Update the last pivot
                  last_pivot_index = i
              else:
                  # Remove the current lower pivot
                  df.loc[df.index[i], 'Lower_Pivot'] = 0
          else:
              # Update pivot type and index
              pivot_type = 'lower'
              last_pivot_index = i
    return df

In [6]:
data =add_pivots(data, 10)

In [7]:
def plot_pivots(data):
  candlestick = go.Candlestick(
      x=data.index,
      open=data['Open'],
      high=data['High'],
      low=data['Low'],
      close=data['Close'],
      name='Candlestick'
  )

  # Step 3: Add markers for higher and lower pivots
  higher_pivots_df = data[data['Higher_Pivot'] == 1]
  lower_pivots_df = data[data['Lower_Pivot'] == 1]

  higher_pivots = go.Scatter(
      x=higher_pivots_df.index,
      y=higher_pivots_df['High'],
      mode='markers',
      marker=dict(color='green', size=10, symbol='triangle-down'),
      name='Higher Pivot'
  )

  lower_pivots = go.Scatter(
      x=lower_pivots_df.index,
      y=lower_pivots_df['Low'],
      mode='markers',
      marker=dict(color='red', size=10, symbol='triangle-up'),
      name='Lower Pivot'
  )

  # Step 4: Create the figure
  fig = go.Figure(data=[candlestick, higher_pivots, lower_pivots])

  # Update layout
  fig.update_layout(
      title='Candlestick Chart with Higher and Lower Pivots',
      xaxis_title='Date',
      yaxis_title='Price',
      xaxis_rangeslider_visible=False,
      template='plotly_dark'
  )

  # Show the plot
  fig.show()

In [8]:
plot_pivots(data)

In [9]:
data['Pivot'] = np.where(data['Higher_Pivot'] == 1, -1, np.where(data['Lower_Pivot'] == 1, 1, 0))
del data['Higher_Pivot']
del data['Lower_Pivot']
data

<ipython-input-9-4b01a0a1c0f2>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Open,High,Low,Close,Volume,Pivot
Date,,,,,,
2024-01-01 00:00:00,42283.58,42775.00,42230.08,42330.49,3948.08335,0
2024-01-01 04:00:00,42330.50,42500.00,42180.77,42492.46,2706.99880,0
2024-01-01 08:00:00,42492.46,42762.39,42452.58,42690.20,2948.80274,0
2024-01-01 12:00:00,42690.21,42847.07,42580.00,42783.05,2936.44406,0
2024-01-01 16:00:00,42783.05,43550.00,42664.42,43517.99,5686.97164,0
...,...,...,...,...,...,...
2024-12-30 08:00:00,93731.66,94068.56,93469.56,93923.14,1683.90269,0
2024-12-30 12:00:00,93923.14,94277.23,91530.45,92130.21,9295.96250,0
2024-12-30 16:00:00,92130.21,94684.09,91800.00,94417.04,6130.98017,0


In [10]:
# Add technical indicators
data.ta.rsi(length=14, append=True)  # RSI
data.ta.sma(length=20, append=True)  # 20-day SMA
data.ta.macd(append=True)  # MACD
data.ta.ema(length=50, append=True)  # 50-day EMA

<ipython-input-10-da7734a157d0>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-da7734a157d0>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-da7734a157d0>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-da7734a

,EMA_50
Date,
2024-01-01 00:00:00,NaN
2024-01-01 04:00:00,NaN
2024-01-01 08:00:00,NaN
2024-01-01 12:00:00,NaN
2024-01-01 16:00:00,NaN
...,...
2024-12-30 08:00:00,95965.709740
2024-12-30 12:00:00,95815.297985
2024-12-30 16:00:00,95760.464339


In [11]:
data

,Open,High,Low,Close,Volume,Pivot,RSI_14,SMA_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,EMA_50
Date,,,,,,,,,,,,
2024-01-01 00:00:00,42283.58,42775.00,42230.08,42330.49,3948.08335,0,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 04:00:00,42330.50,42500.00,42180.77,42492.46,2706.99880,0,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 08:00:00,42492.46,42762.39,42452.58,42690.20,2948.80274,0,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 12:00:00,42690.21,42847.07,42580.00,42783.05,2936.44406,0,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 16:00:00,42783.05,43550.00,42664.42,43517.99,5686.97164,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-30 08:00:00,93731.66,94068.56,93469.56,93923.14,1683.90269,0,40.897906,94645.1710,-671.536841,-47.117604,-624.419237,95965.709740
2024-12-30 12:00:00,93923.14,94277.23,91530.45,92130.21,9295.96250,0,31.693500,94496.4435,-797.456429,-138.429754,-659.026676,95815.297985
2024-12-30 16:00:00,92130.21,94684.09,91800.00,94417.04,6130.98017,0,47.823230,94369.6765,-704.598370,-36.457355,-668.141014,95760.464339


In [12]:
# Drop rows with NaN values (due to rolling calculations)
data.dropna(inplace=True)

<ipython-input-12-b4c9a74398dc>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
# Step 3: Prepare data for LSTM
def create_sequences(data, target_col, seq_length):
    """
    Create sequences of `seq_length` time steps for LSTM input.
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i + seq_length].drop(columns=[target_col]).values)
        y.append(data.iloc[i + seq_length][target_col])
    return np.array(X), np.array(y)

In [14]:
seq_length = 15
target_col = 'Pivot'

# Create sequences
X, y = create_sequences(data, target_col, seq_length)

In [27]:
X.shape

(2127, 15, 11)

In [28]:
y.shape

(2127, 3)

In [23]:
# Encode target variable (pivot) as categorical
y = tf.keras.utils.to_categorical(y + 1, num_classes=3)  # -1 -> 0, 0 -> 1, 1 -> 2

# Normalize features
scaler = MinMaxScaler()
X = np.array([scaler.fit_transform(x) for x in X])

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [32]:
y

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [33]:
# Step 4: Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(3, activation='softmax')  # Output layer for 3 classes (-1, 0, 1)
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [34]:
# Step 5: Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 49ms/step - accuracy: 0.8989 - loss: 0.5261 - val_accuracy: 0.9554 - val_loss: 0.2153
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9657 - loss: 0.1858 - val_accuracy: 0.9554 - val_loss: 0.2167
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9443 - loss: 0.2614 - val_accuracy: 0.9554 - val_loss: 0.2144
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9425 - loss: 0.2667 - val_accuracy: 0.9554 - val_loss: 0.2132
Epoch 5/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.9503 - loss: 0.2345 - val_accuracy: 0.9554 - val_loss: 0.2138
Epoch 6/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9435 - loss: 0.2625 - val_accuracy: 0.9554 - val_loss: 0.2131
Epoch 7/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9469 - loss: 0.2486 - val_accuracy: 0.9554 - val_loss: 0.2140
Epoch 8/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.9503 - loss: 0.2312 - val_accuracy: 0.9554 - v

In [35]:
# Step 6: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Test Loss: 0.1996
Test Accuracy: 0.9554


,Open,High,Low,Close,Volume,Pivot,RSI_14,SMA_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,EMA_50
Date,,,,,,,,,,,,
2024-01-09 04:00:00,46926.96,46940.00,46599.43,46825.51,4707.40981,0,74.700090,44603.7810,712.503031,349.412947,363.090085,44148.235000
2024-01-09 08:00:00,46825.52,46874.94,46381.10,46615.99,6018.83491,0,71.207769,44738.0255,748.547491,308.365925,440.181566,44245.009706
2024-01-09 12:00:00,46616.00,47051.21,45450.00,46868.09,15811.82358,0,72.852357,44898.7700,788.367538,278.548778,509.818760,44347.875600
2024-01-09 16:00:00,46868.08,46975.00,46500.00,46876.01,7310.48299,0,72.904719,45058.9725,811.213130,241.115496,570.097634,44447.018125
2024-01-09 20:00:00,46876.00,47972.00,44748.67,46110.00,25976.98557,0,60.708530,45171.1140,758.761293,150.930927,607.830366,44512.233101
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-30 08:00:00,93731.66,94068.56,93469.56,93923.14,1683.90269,0,40.897906,94645.1710,-671.536841,-47.117604,-624.419237,95965.709740
2024-12-30 12:00:00,93923.14,94277.23,91530.45,92130.21,9295.96250,0,31.693500,94496.4435,-797.456429,-138.429754,-659.026676,95815.297985
2024-12-30 16:00:00,92130.21,94684.09,91800.00,94417.04,6130.98017,0,47.823230,94369.6765,-704.598370,-36.457355,-668.141014,95760.464339


In [37]:
# Step 7: Predict pivots
data['Predicted_Pivot'] = np.nan
for i in range(seq_length, len(data)):
    input_data = scaler.transform(data.iloc[i - seq_length:i].drop(columns=[target_col, 'Predicted_Pivot']))
    input_data = np.expand_dims(input_data, axis=0)
    prediction = model.predict(input_data, verbose=0)
    predicted_class = np.argmax(prediction) - 1  # Convert back to -1, 0, 1
    data.loc[data.index[i], 'Predicted_Pivot'] = predicted_class

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning:

X has feature names, but MinMaxScaler was fitted without feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning:

X has feature names, but MinMaxScaler was fitted without feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning:

X has feature names, but MinMaxScaler was fitted without feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning:

X has feature names, but MinMaxScaler was fitted without feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning:

X has feature names, but MinMaxScaler was fitted without feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning:

X has feature names, but MinMaxScaler was fitted without feat

In [38]:
data

,Open,High,Low,Close,Volume,Pivot,RSI_14,SMA_20,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,EMA_50,Predicted_Pivot
Date,,,,,,,,,,,,,
2024-01-09 04:00:00,46926.96,46940.00,46599.43,46825.51,4707.40981,0,74.700090,44603.7810,712.503031,349.412947,363.090085,44148.235000,NaN
2024-01-09 08:00:00,46825.52,46874.94,46381.10,46615.99,6018.83491,0,71.207769,44738.0255,748.547491,308.365925,440.181566,44245.009706,NaN
2024-01-09 12:00:00,46616.00,47051.21,45450.00,46868.09,15811.82358,0,72.852357,44898.7700,788.367538,278.548778,509.818760,44347.875600,NaN
2024-01-09 16:00:00,46868.08,46975.00,46500.00,46876.01,7310.48299,0,72.904719,45058.9725,811.213130,241.115496,570.097634,44447.018125,NaN
2024-01-09 20:00:00,46876.00,47972.00,44748.67,46110.00,25976.98557,0,60.708530,45171.1140,758.761293,150.930927,607.830366,44512.233101,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-30 08:00:00,93731.66,94068.56,93469.56,93923.14,1683.90269,0,40.897906,94645.1710,-671.536841,-47.117604,-624.419237,95965.709740,0.0
2024-12-30 12:00:00,93923.14,94277.23,91530.45,92130.21,9295.96250,0,31.693500,94496.4435,-797.456429,-138.429754,-659.026676,95815.297985,0.0
2024-12-30 16:00:00,92130.21,94684.09,91800.00,94417.04,6130.98017,0,47.823230,94369.6765,-704.598370,-36.457355,-668.141014,95760.464339,0.0


In [39]:
data['Predicted_Pivot'].value_counts()
# data['Pivot'].value_counts()

,count
Predicted_Pivot,
-1.0,1819
0.0,294
1.0,14


In [40]:
def plot_pivots_2(data):
  candlestick = go.Candlestick(
      x=data.index,
      open=data['Open'],
      high=data['High'],
      low=data['Low'],
      close=data['Close'],
      name='Candlestick'
  )

  # Step 3: Add markers for higher and lower pivots
  higher_pivots_df = data[data['Predicted_Pivot'] == -1]
  lower_pivots_df = data[data['Predicted_Pivot'] == 1]

  higher_pivots = go.Scatter(
      x=higher_pivots_df.index,
      y=higher_pivots_df['High'],
      mode='markers',
      marker=dict(color='red', size=10, symbol='triangle-down'),
      name='Higher Pivot'
  )

  lower_pivots = go.Scatter(
      x=lower_pivots_df.index,
      y=lower_pivots_df['Low'],
      mode='markers',
      marker=dict(color='green', size=10, symbol='triangle-up'),
      name='Lower Pivot'
  )

  # Step 4: Create the figure
  fig = go.Figure(data=[candlestick, higher_pivots, lower_pivots])

  # Update layout
  fig.update_layout(
      title='Candlestick Chart with Predicted Higher and Lower Pivots',
      xaxis_title='Date',
      yaxis_title='Price',
      xaxis_rangeslider_visible=False,
      template='plotly_dark'
  )

  # Show the plot
  fig.show()

In [41]:
plot_pivots_2(data)